In [2]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00


In [4]:
import os

import pandas as pd
import pytube
from pytube.exceptions import PytubeError
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
video_path = '/content/drive/MyDrive/Colab Notebooks/Курс компьютерное зрение/videos/'
data_path = '/content/drive/MyDrive/Colab Notebooks/Курс компьютерное зрение/data/'

In [8]:
df = pd.read_csv('train.csv')

In [9]:
df = df[df.label.str.contains('dancing')]

In [10]:
df

,label,youtube_id,time_start,time_end,split
17,tap dancing,--6q_33gNew,132,142,train
142,breakdancing,-0AFA2OVzn0,19,29,train
166,belly dancing,-0L4z8D2eGs,43,53,train
179,tap dancing,-0SUZKVInwE,81,91,train
298,dancing charleston,-1KtlEOx78I,9,19,train
...,...,...,...,...,...
532751,tango dancing,zyN5s4RTthw,175,185,train
532759,tango dancing,zyOZVfNrbX0,120,130,train
532760,dancing ballet,zyOb-PosWSY,68,78,train
532772,tango dancing,zyY6SLwKvrk,1,11,train


In [13]:
new_df = pd.DataFrame()
label_values = df['label'].unique()
for label in label_values:
    label_data = df[df['label'] == label].sample(20)
    new_df = pd.concat([new_df, label_data])


In [14]:
new_df

,label,youtube_id,time_start,time_end,split
9335,tap dancing,0GQFYsxMP9Y,53,63,train
396539,tap dancing,k7TUjQcSFTk,44,54,train
48132,tap dancing,5MiwOATS6dY,18,28,train
89923,tap dancing,AgKxqjEu-4Y,171,181,train
361832,tap dancing,g1NjvzdaSUI,0,10,train
...,...,...,...,...,...
94463,tango dancing,BGJw-UnMV44,290,300,train
117929,tango dancing,EEHupJ8g_jI,101,111,train
511332,tango dancing,xJwn3eI3yCY,66,76,train
518981,tango dancing,yFe_5H_bwsM,18,28,train


In [ ]:
new_df = new_df.reset_index()
k = 0
length = new_df.shape[0]
lst_name_video = []
lst_target = []
for i, row in new_df.iterrows():
    try:
        tag = row['youtube_id']
        video_url = f"https://www.youtube.com/watch?v={tag}"
        yt = pytube.YouTube(video_url)
        stream = yt.streams.first()
        filename = "../video.mp4"
        stream.download(filename=filename)

        start_time = row.time_start
        end_time = row.time_end

        name_video = f"{video_path}video_{k:04d}.mp4"
        ffmpeg_extract_subclip(filename, start_time, end_time, targetname=name_video)
        k += 1
        lst_name_video.append(name_video)
        lst_target.append(row.label)
    except PytubeError as e:
        print(f"Error occurred while processing video {i + 1}: {str(e)}")
    except Exception as e:
        print(f"Unexpected error occurred while processing video {i + 1}: {str(e)}")

    print(f"{i + 1} / {length} is ready")

data_n = pd.DataFrame({"name_video": lst_name_video,
                       "label": lst_target})

In [21]:
data_n.to_csv(os.path.join(data_path, 'data.csv'), index=False)

In [18]:
data_n

,name_video,label
0,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
1,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
2,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
3,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
4,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
...,...,...
205,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing
206,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing
207,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing
208,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing
